#  MODIS Resampling  
## Subsection 1a

Continuation of "Resampling MODIS" section in the [1]ActiveFires_California.ipynb 

In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.overwriteoutput = True

In [ ]:
# Connect to google drive directory as G: Drive in local desktop environment with arcpy installed 
rootPath = "G:\My Drive\California FireTrends (2012-2020)"
env.workspace = rootPath

In [ ]:
import arcgisscripting
gp = arcgisscripting.create()
gp.CheckOutExtension("Spatial")
gp.overwriteoutput = True

os.chdir(rootPath)

In [ ]:
# get list of files based on directory and extension inputs 
def listFiles_ByExt(rootPath, ext):
    file_list = []
    root = rootPath
    for path, subdirs, files in os.walk(root):
        for names in files: 
            if names.endswith(ext) and not names.startswith("._"):
                file_list.append(os.path.join(path,names))
    return(file_list)

# Create new folder in root path 
def createFolder(rootPath, folderName): 
    folderPath = os.path.join(rootPath, folderName) 
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    return folderPath + "\\" 

In [ ]:
# get all filenames for M6 Active fire shapefiles for each day in yr (created in Colab notebook)
M6_1km_path = listFiles_ByExt('Data\MODIS_Resampling\SHP', '.shp')

# from all pathnames, we extract 2020 files to perform update
recent_2020_path = [e for e in M6_1km_path if '2020' in e]

In [ ]:
# check if all files were read 
recent_2020_path.sort()

In [ ]:
# check spatial reference is projected 
arcpy.Describe(M6_1km_path[0]).spatialReference

In [ ]:
def resampling(file_nm, attribute_ID, cell, Value):
    '''
    Original MODIS (M6) AF points of each unique day in year are represented as 1km raster grids, 
    which are then resampled to 375m and converted back to vector points. 
    '''
    v6_name = m6.replace("SHP", "RAS")
    v6_name = v6_name.replace(".shp", "_" + attribute_ID + ".tif")
    resamp_v6_name = v6_name.replace("RAS", "RESAMP_RAS")
    pnt_v6_name = resamp_v6_name.replace("RESAMP_RAS", "RESAMP_SHP")
    pnt_v6_name = pnt_v6_name.replace(".tif", ".shp")
    
    if os.path.exists(pnt_v6_name) == False:
        if not os.path.exists(os.path.dirname(v6_name)):
            os.makedirs(os.path.dirname(v6_name))
            
        sr = arcpy.SpatialReference(3310)
        arcpy.DefineProjection_management(m6, sr)
        print("converting pnt to ras:")
        ras = arcpy.PointToRaster_conversion(file_nm, attribute_ID, v6_name, 'MOST_FREQUENT', '', '1000') #0.009

        print("resampling to 375m:", v6_name)

        if not os.path.exists(os.path.dirname(resamp_v6_name)):
            os.makedirs(os.path.dirname(resamp_v6_name))

        resamp_ras = arcpy.Resample_management(ras, resamp_v6_name, "375", "NEAREST") #0.003378

        print("converting ras to pnt:", resamp_v6_name)

        if not os.path.exists(os.path.dirname(pnt_v6_name)):
            os.makedirs(os.path.dirname(pnt_v6_name))

        resamp_pnts = arcpy.RasterToPoint_conversion(resamp_ras, pnt_v6_name, Value)
        
        return resamp_pnts

In [ ]:
# reverse list to perform resampling on days that have not yet been added yet 
# resample both Julian day and time so that earliest time detected can be stored 
for i, m6 in enumerate(reversed(recent_2020_path)):  
    print(i+1, ' of ', len(recent_2020_path), ' complete')
    jd_pnts = resampling(m6, "JD", "MINIMUM", 'Value')
    time_pnts = resampling(m6, "Time", "MINIMUM", 'Time')
        
    outfc =  m6.replace("SHP", "RESAMP_SHP_DT")
    if os.path.exists(outfc) == False:
        outpth  = outfc.split('\\')
        outpth = '\\'.join(outpth[:-1])
        if not os.path.exists(outpth):
            os.makedirs(outpth)
        arcpy.SpatialJoin_analysis(jd_pnts, time_pnts, outfc)